# Install libs

In [1]:
try:
  from icecream import ic
except Exception:
  !pip install -q icecream --no-index --find-links=file:///kaggle/input/icecream

# Import libs

In [2]:
import sys, os
import numpy as np
import pandas as pd
import json
import re
import six
import glob
import traceback
import inspect
from typing import Union
from collections import Counter, OrderedDict, defaultdict
from collections.abc import Iterable
from multiprocessing import cpu_count
from IPython.display import display
from tqdm.notebook import tqdm
from icecream import ic
import transformers
import tensorflow as tf
import torch
from torch import nn, einsum
import torch.nn.functional as F
ic(tf.__version__, torch.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

('2.12.0', '2.0.0+cpu')

# Force tf not use gpu memory, for only use tf for tfrecord reading,preprocessing and postprocessing, not for train(using torch)

In [3]:
tf.config.set_visible_devices([], 'GPU')

# Flags

In [4]:
class FLAGS(object):
  # for tfrecords args, you could ignore
  seed = 1024
  batch_parse = False
  sparse_to_dense = True
  eval_keys = []
  incl_keys = []
  excl_keys = []
  recount_tfrecords = False  
  batch_sizes = []
  buffer_size = 1024
  buckets = None
  drop_remainder = None
  shard_by_files = True
  shuffle_batch = None
  shuffle_files = None
  num_dataset_threads = 0
  num_prefetch_batches = 1024
  repeat_then_shuffle = False
  length_index = 1
  length_key = None
  dynamic_pad = True
  cache = False
  cache_after_map = False
  fixed_random = False
  parallel_read_files = True
  padding_idx = 0
  dataset_keys = []
  dataset_excl_keys = []
  exclude_varlen_keys = False
  prefetch = None
  dataset_ordered = False
    
  torch = True
  keras = False
    
  # online==False means using n-fold split and train on fold 1,2, folds-1 while valid on fold 0
  # online==True means using all train data but still will valid on fold 0
  online = False  
  folds = 4
  fold = 0
  fold_seed = 1229
  root = '../input/asl-fingerspelling'
  working = '/kaggle/working'
  use_z = True  # use x,y,z if True
  norm_frames = True # norm frames using x - mean / std
  concat_frames = True # concat original and normalized frames
  add_pos = True # add abs frame pos, like 1/1000., 2/1000.
  sup_weight = 0.1 # for supplement dataset assigin weight 0.1
  
  train_files = []
  valid_files = []
      
  mix_sup = True # train & sup dataset
  vie = 5 # valid interval epochs 
  lr = 2e-3
  epochs = 400 
  batch_size = 128
  eval_batch_size = 256
  awp = True
  adv_start_epoch = None
  adv_lr = 0.2
  adv_eps = 0
  fp16 = False # notice fp16 could not be set True if using awp here, otherwise nan
  optimizer = 'Adam'
  opt_eps = 1e-6 
  scheduler = 'linear'
  # for model related configs
  encoder_layers = 17
  encoder_units = 200 
  n_frames = 320  
  distributed = False

  # for preprocess
  trunct_method = 'resize'

  # for aug
  flip_rate = 0.25
  resample_rate = 0.8
  temporal_mask_rate = 0.8
  temporal_mask_prob = 0.5
  temporal_mask_range = [0.1, 0.5]
  spatio_mask_rate = 0.
  spatio_mask_prob = 0.
  add_pos_before_resample = True
  shift_rate = 0.
  shift_range = [-0.05, 0.05]
  shift_method = 1
  temporal_seq_mask_rate = 0.5
  temporal_seq_mask_max = 2
  temporal_seq_mask_range = [0.1, 0.2]
  shift_rate = 0.75
  scale_method = 0
  scale_rate = 0.75
  rotate_rate = 0.75
  
  # for model
  ksize_vals = [15]
  skip_factor = 0.5
  mhatt_heads = 8
  mhatt_dimhead = 32

FLAGS.adv_start_epoch = int(FLAGS.epochs * 0.15)
    
def load_json(filename):
  with open(filename) as fh:
    obj = json.load(fh)
  return obj

In [5]:
FLAGS.batch_size = 128
FLAGS.acc_steps = 4 # gradient acc
FLAGS.batch_size = FLAGS.batch_size // FLAGS.acc_steps

# Common configs

In [6]:
LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

LIP = [
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
ic(len(LIP))
LLIP = [84,181,91,146,61,185,40,39,37,87,178,88,95,78,191,80,81,82]
RLIP = [314,405,321,375,291,409,270,269,267,317,402,318,324,308,415,310,311,312]
MID_LIP = [i for i in LIP if i not in LLIP + RLIP]
ic(len(LLIP), len(RLIP), len(MID_LIP))

NOSE=[
    1,2,98,327
]
LNOSE = [98]
RNOSE = [327]
MID_NOSE = [i for i in NOSE if i not in LNOSE + RNOSE]

LEYE = [
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    466, 388, 387, 386, 385, 384, 398,
]
REYE = [
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    246, 161, 160, 159, 158, 157, 173,
]

N_HAND_POINTS = 21
N_POSE_POINTS = len(LPOSE)
N_LIP_POINTS = len(LLIP)
N_EYE_POINTS = len(LEYE)
N_NOSE_POINTS = len(LNOSE)
N_MID_POINTS = len(MID_LIP + MID_NOSE)

SEL_COLS = []
for i in range(N_HAND_POINTS):
  SEL_COLS.extend([f'x_left_hand_{i}', f'y_left_hand_{i}', f'z_left_hand_{i}'])
for i in range(N_HAND_POINTS):
  SEL_COLS.extend([f'x_right_hand_{i}', f'y_right_hand_{i}', f'z_right_hand_{i}'])
for i in LPOSE:
  SEL_COLS.extend([f'x_pose_{i}', f'y_pose_{i}', f'z_pose_{i}'])
for i in RPOSE:
  SEL_COLS.extend([f'x_pose_{i}', f'y_pose_{i}', f'z_pose_{i}'])
for i in LLIP:
  SEL_COLS.extend([f'x_face_{i}', f'y_face_{i}', f'z_face_{i}'])
for i in RLIP:
  SEL_COLS.extend([f'x_face_{i}', f'y_face_{i}', f'z_face_{i}'])

for i in LEYE:
  SEL_COLS.extend([f'x_face_{i}', f'y_face_{i}', f'z_face_{i}'])
for i in REYE:
  SEL_COLS.extend([f'x_face_{i}', f'y_face_{i}', f'z_face_{i}'])
  
for i in LNOSE:
  SEL_COLS.extend([f'x_face_{i}', f'y_face_{i}', f'z_face_{i}'])
for i in RNOSE:
  SEL_COLS.extend([f'x_face_{i}', f'y_face_{i}', f'z_face_{i}'])
  
for i in MID_LIP:
  SEL_COLS.extend([f'x_face_{i}', f'y_face_{i}', f'z_face_{i}'])
for i in MID_NOSE:
  SEL_COLS.extend([f'x_face_{i}', f'y_face_{i}', f'z_face_{i}'])
    
N_COLS = len(SEL_COLS)
ic(N_COLS)
    
CHAR2IDX = load_json(f'../input/asl-fingerspelling/character_to_prediction_index.json')
CHAR2IDX = {k: v + 1 for k, v in CHAR2IDX.items()}
N_CHARS = len(CHAR2IDX)
ic(N_CHARS)

PAD_IDX = 0
SOS_IDX = PAD_IDX # Start Of Sentence
EOS_IDX = N_CHARS + 1 # End Of Sentence
ic(PAD_IDX, SOS_IDX, EOS_IDX)

PAD_TOKEN = '<PAD>'
SOS_TOKEN = PAD_TOKEN
EOS_TOKEN = '<EOS>'

CHAR2IDX[PAD_TOKEN] = PAD_IDX
CHAR2IDX[EOS_TOKEN] = EOS_IDX 

ADDRESS_TOKEN = '<ADDRESS>'
URL_TOKEN = '<URL>'
PHONE_TOKEN = '<PHONE>'
SUP_TOKEN = '<SUP>'

VOCAB_SIZE = len(CHAR2IDX)
IDX2CHAR = {v: k for k, v in CHAR2IDX.items()}
ic(VOCAB_SIZE)
ic(len(IDX2CHAR))

STATS = {}
CLASSES = [
  'address', 
  'url', 
  'phone', 
  'sup',
  ]
PHRASE_TYPES = dict(zip(CLASSES, range(len(CLASSES))))
N_TYPES = len(CLASSES)
MAX_PHRASE_LEN = 32

def get_phrase_type(phrase):
  # Phone Number
  if re.match(r'^[\d+-]+$', phrase):
    return 'phone'
  # url
  elif any([substr in phrase for substr in ['www', '.', '/']
           ]) and ' ' not in phrase:
    return 'url'
  # Address
  else:
    return 'address'

def get_vocab_size():
  vocab_size = VOCAB_SIZE
  return vocab_size

def get_n_cols(no_motion=False, use_z=None):
  n_cols = N_COLS
  if use_z is None:
    use_z = FLAGS.use_z
  
  if FLAGS.concat_frames:
    assert FLAGS.norm_frames
    n_cols += N_COLS
  
  if not use_z:
    n_cols = n_cols // 3 * 2
    
  if FLAGS.add_pos:
    n_cols += 1
  
  return n_cols

ic| len(LIP): 40
ic| len(LLIP): 18, len(RLIP): 18, len(MID_LIP): 4
ic| N_COLS: 384
ic| N_CHARS: 59
ic| PAD_IDX: 0, SOS_IDX: 0, EOS_IDX: 60
ic| VOCAB_SIZE: 61
ic| len(IDX2CHAR): 61


In [ ]:
"""
    Tensorflow layer to process data in TFLite
    Data needs to be processed in the model itself, so we can not use Python
    [None, N_COLS] -> [1, NONE, N_COLS] -> [1, N_FRAMES, N_COLS]
"""
class PreprocessLayer(tf.keras.layers.Layer):

  def __init__(self, n_frames=128, training=False):
    super(PreprocessLayer, self).__init__()
    self.n_frames = n_frames
    self.n_cols = get_n_cols(no_motion=True, use_z=True)
    ic(self.n_cols)
    self.means = np.load(f'../input/3rd-place-step-3-gen-mean-and-std/means.npy')
    self.stds = np.load(f'../input/3rd-place-step-3-gen-mean-and-std/stds.npy')
    ic(self.means.shape, self.stds.shape)
    self.training = training

  ## seems not needed as training perf is similar
  @tf.function(input_signature=(tf.TensorSpec(shape=[None, N_COLS],
                                              dtype=tf.float32),),)
  ## this will cause error
  # @tf.function()
  def call(self, data):
    trunct_method = FLAGS.trunct_method
    training = self.training

    # Hacky (add dim in front line shape [None,N_COLS] to shape [1, None, N_COLS])
    data = data[None]
    dtype = data.dtype
    
    # [1, None, N_COLS//3, 3]
    data = reshape(data)
    
    if training and FLAGS.use_aug:
      data = Apply(scale, FLAGS.scale_rate)(data)
      data = Apply(rotate, FLAGS.rotate_rate)(data)
      # shift not affect much..
      data = Apply(shift, FLAGS.shift_rate)(data)
      data = OneOf([scale, rotate, shift], FLAGS.affine_rate)(data)
      data = Apply(temporal_seq_mask, FLAGS.temporal_seq_mask_rate)(data)
    
    data_ = data
    data = reshape_back(data)
      
    # seems norm before resize produce better results
    # -------norm frames
    l = []
    if FLAGS.norm_frames:
      if FLAGS.concat_frames:
        l.append(data_)
      l.append(
          (data - self.means) / self.stds,
      )
      l[-1] = reshape(l[-1])
    else:
      l.append(data_)
            
    # [1, None, feats, 3]
    data = tf.concat(l, axis=-2)

    n_cols = self.n_cols if not FLAGS.add_pos else self.n_cols - 1
    data = tf.reshape(data, [1, -1, n_cols])
    # Fill NaN Values With 0
    data = tf.where(tf.math.is_nan(data), tf.zeros_like(data), data)
                       
    # add_pos helps a lot and add_pos_before_resample also +0.002 compare to add_pos_after_resample
    if FLAGS.add_pos and FLAGS.add_pos_before_resample:
      # n_frames = len(data[0])
      n_frames = tf.shape(data)[1]
      pos = get_pos(n_frames, dtype)
      data = tf.concat([data, pos], axis=-1)
      
    #  now basic features done, we resample for frames with basic features
    if training and FLAGS.use_aug:
      # resample for time range, like 120 frames to 135 or 90 frames
      # resample helps a lot
      data = Apply(resample, FLAGS.resample_rate)(data)
      data = tf.cast(data, dtype)
      
    # ------add addional info like add pos after resample, actually similar results before or after resample
    # n_frames = len(data[0])
    n_frames = tf.shape(data)[1]
    
    # do it after expand dim 0, and do not use expand_dims before...
    if FLAGS.add_pos and (not FLAGS.add_pos_before_resample):
      pos = get_pos(n_frames, dtype)
      data = tf.concat([data, pos], axis=-1)

    if not FLAGS.always_resize:
      # Pad Zeros  TODO dynamic type so not pad? dynanmic seq2seq input
      # n_frames = len(data[0])
      n_frames = tf.shape(data)[1]
      if n_frames < self.n_frames:
        if FLAGS.pad_frames:
          if FLAGS.pad_method == 'zero':
            data = tf.concat((data,
                              tf.zeros([1, self.n_frames - n_frames, self.n_cols],
                                        dtype=dtype)),
                            axis=1)
          else:
            data = tf.image.resize(
              data,
              [1, self.n_frames],
              method=FLAGS.pad_resize_method,
            )
        elif n_frames < FLAGS.encode_pool_size:
          data = tf.concat(
              (data,
               tf.zeros([1, FLAGS.encode_pool_size - n_frames, self.n_cols],
                        dtype=dtype)),
              axis=1)
        elif n_frames < FLAGS.min_frames:
          data = tf.concat(
              (data,
               tf.zeros([1, FLAGS.min_frames - n_frames, self.n_cols],
                        dtype=dtype)),
              axis=1)

      # n_frames = len(data[0])
      n_frames = tf.shape(data)[1]
      if n_frames > self.n_frames:
        if trunct_method == 'resize':
          # Downsample
          data = tf.image.resize(
              data,
              [1, self.n_frames],
              method=FLAGS.resize_method,
          )
          # For resize The return value has type float32, unless the method is ResizeMethod.NEAREST_NEIGHBOR
          data = tf.cast(data, dtype)
        else:
          data = data[:, :self.n_frames, :]
    else:
      data = tf.image.resize(
          data,
          [1, self.n_frames],
          method=FLAGS.resize_method,
      )
      data = tf.cast(data, dtype)

    if FLAGS.pad_frames:
      data = tf.reshape(data, [1, self.n_frames, self.n_cols])
      

    if training and FLAGS.use_aug:
      # temperal mask help a lot
      data = Apply(temporal_mask, FLAGS.temporal_mask_rate)(data)
      data = Apply(spatio_mask, FLAGS.spatio_mask_rate)(data)
      
    if not FLAGS.use_z:
      if FLAGS.add_pos:
        pos_data = data[...,-2:-1]
        data = data[...,:-1]
      data = tf.reshape(data, [1, self.n_frames, -1, 3])
      data = data[...,:2]
      data = tf.reshape(data, [1, self.n_frames, -1])
      if FLAGS.add_pos:
        data = tf.concat([data, pos_data], axis=-1)
             
    # Squeeze Batch Dimension
    data = tf.squeeze(data, axis=[0])
    # tf.print('----', data.shape)
    return data

class PreProcssor(object):
  def __init__(self, subset='valid', squeeze=False):
    training = subset == 'train'
    self.prepocess = PreprocessLayer(FLAGS.n_frames, training=training)
    self.squeeze = squeeze

  def __call__(self, fe):
    # # TODO HACK for hug datasets.. ds = ds.to_tf_dataset(batch_size=1)
    if self.squeeze:
      for key in fe:
        fe[key] = tf.squeeze(fe[key], axis=0)
    x = fe
    weights = fe['weight']
    fe['frames'] = tf.reshape(fe['frames'], [fe['n_frames'], N_COLS])
    fe['frames'] = self.prepocess(fe['frames']) 
    fe['phrase_len'] = tf.cond(tf.greater(fe['phrase_len'], MAX_PHRASE_LEN), lambda: tf.constant(MAX_PHRASE_LEN, fe['phrase_len'].dtype), lambda: fe['phrase_len'])
    if FLAGS.no_eos:
      # as in tfrecords we have eos, but in training if we don't have eos we change it to pad
      mask = tf.logical_or(fe['phrase_'] == PAD_IDX, fe['phrase_'] == EOS_IDX)
      mask = tf.cast(mask, fe['phrase_'].dtype)
      fe['phrase_'] = fe['phrase_'] * (1 - mask) + mask * PAD_IDX
        
    if FLAGS.mix_sup:
      weights_mask = tf.cast(weights != 1, tf.float32)
      weights = weights_mask * FLAGS.sup_weight + (1 - weights_mask)
      x['weight'] = weights
    
    y = fe['phrase_']
    return x, y
  

In [8]:
FLAGS.use_aug = True
FLAGS.always_resize = False
FLAGS.affine_rate = 0
FLAGS.force_flip = False
FLAGS.scale_rate = 0.75
FLAGS.scale_range = [0.8, 1.2]
FLAGS.rotate_range = [-15.0, 15.0]
FLAGS.shift_range = [-0.05, 0.05]
FLAGS.temporal_seq_mask_range = [0.1, 0.2]
FLAGS.pad_frames = True
FLAGS.pad_method = 'zero'
FLAGS.resize_method = 'bilinear'
FLAGS.no_eos = False

In [9]:
from torch.utils.data import IterableDataset

class TorchDataset(IterableDataset):
  def __init__(self, subset='eval'):
    self.subset = subset

    if subset == 'train':
      dataset = TfDataset('train', files=FLAGS.train_files)
      datas = dataset.make_batch(FLAGS.batch_size, 
                             shuffle=True,
                             repeat=True,
                             drop_remainder=True,
                             return_numpy=True)
    else:
      dataset = TfDataset('valid', files=FLAGS.valid_files)
      datas = dataset.make_batch(FLAGS.eval_batch_size,  
                             shuffle=False,
                             repeat=False, 
                             drop_remainder=False,
                             return_numpy=False) # if set return numpy = True then you could only visit 1 time..
    
    self.num_instances = dataset.num_instances
    ic(self.num_instances)
    self.num_steps = dataset.num_steps
    ic(self.num_steps)
    
    self.datas = datas
    self.data_iter = iter(datas)
         
  def __iter__(self):
    if self.subset == 'train':
      while True:
        x, y = next(self.data_iter)
        del x['phrase_type']
        del x['phrase']
        if FLAGS.distributed:
          rank = gezi.get('RANK', 0)
          start = rank * FLAGS.batch_size
          end = start + FLAGS.batch_size
          for key in x:
            x[key] = x[key][start:end]
          y = y[start:end]
        yield x, y
    else:
      for x, y in self.datas:
        for key in x:
          x[key] = x[key].numpy()
        y = y.numpy()
        yield x, y
    
  def __len__(self):
    return self.num_instances
  
def get_dataloaders():
  kwargs = {
      'num_workers': 0, # >0 will hang...
      'pin_memory': True,
      'persistent_workers': False,
  }
  
  train_ds = TorchDataset('train')
  eval_ds = TorchDataset('eval')
  
  train_dl = torch.utils.data.DataLoader(train_ds,
                                         batch_size=None,
                                         sampler=None,
                                         **kwargs)
  eval_dl = torch.utils.data.DataLoader(eval_ds,
                                        batch_size=None,
                                        sampler=None,
                                        **kwargs)

  return train_dl, eval_dl  


# Model(17 layers squeezeformer)

In [10]:
FLAGS.encode_pool_size = 2
FLAGS.emb_batchnorm = True
FLAGS.attn_drop = 0
FLAGS.ff_drop = 0
FLAGS.conv_drop = 0
FLAGS.inst_drop_rate = 0.2
FLAGS.cls_drop = 0.1
FLAGS.conv1d_ksize_vals = [15]
FLAGS.conv1d_expansion_factor = 2
FLAGS.ff_mult = 4
FLAGS.encoder_units = 200 
FLAGS.encoder_layers = 17
FLAGS.mhatt_dimhead = 32
FLAGS.mhatt_heads = 8
FLAGS.skip_factor = 0.5
# time reduce from layer 8, means 7 + 10 = 17 layers while the first 7 layers using 320 frames and the last 10 layers using 160 frames
FLAGS.time_reduce = True
FLAGS.time_reduce_idx = 7
FLAGS.time_kernel_size = 5 
FLAGS.time_stride = 2
# for ROPE
FLAGS.scaling_type = 'dynamic'
FLAGS.scaling_factor = 0.5

In [11]:
def reshape(data):
  data = tf.reshape(data, (1, -1, N_COLS // 3, 3))
  return data

def reshape_back(data):
  data = tf.reshape(data, (1, -1, N_COLS))
  return data

def get_pos(n_frames, dtype):
  pos = tf.range(n_frames, dtype=dtype)
  pos /= 1000.
  pos = tf.reshape(pos, [1, n_frames, 1])
  return pos

def simple_decode(pred):
  x = tf.argmax(pred, axis=-1)
  return x

def adjust_pad(pred):
  pred = tf.nn.softmax(pred, axis=-1)
  pred0, pred1 = pred[..., 0:1], pred[..., 1:]
  pred0 *= tf.cast(pred0 > FLAGS.pad_thre, pred.dtype)
  pred = tf.concat([pred0, pred1], axis=-1)
  return pred

def ctc_decode(x):
  x = tf.concat([x, tf.zeros((1), dtype=x.dtype)], axis=0)
  diff = tf.not_equal(x[:-1], x[1:])
  adjacent_indices = tf.where(diff)[:, 0]
  x = tf.gather(x, adjacent_indices)
  mask = x != PAD_IDX
  x = tf.boolean_mask(x, mask, axis=0)
  return x

@tf.function()
def decode_phrase(pred):
  pred = adjust_pad(pred)  
  x = tf.argmax(pred, axis=-1)
  x = ctc_decode(x)
  return x  

In [12]:
class BatchNorm(nn.Module):
  def __init__(self, num_features, momentum=0.1, eps=1e-5):
    super().__init__()
    self.bn = nn.BatchNorm1d(num_features, momentum=momentum, eps=eps)
  
  def forward(self, x):
    x = x.permute(0, 2, 1)
    x = self.bn(x)
    x = x.permute(0, 2, 1)
    return x

class InstanceDropout(nn.Module):
  def __init__(self, p=0.5):
    super().__init__()
    self.p = p
    self.dropout = nn.Dropout(p)

  def forward(self, x):
    mask = torch.ones_like(x[:,:1,:1])
    mask = self.dropout(mask)
    return x * mask

In [13]:
class SimpleEmbedding(nn.Module):

  def __init__(self):
    super().__init__()

    self.emb_batchnorm = True if FLAGS.emb_batchnorm is None else FLAGS.emb_batchnorm
    self.embedding = nn.Linear(get_n_cols(), FLAGS.encoder_units, bias=False)
    if self.emb_batchnorm:
      self.batch_norm = BatchNorm(FLAGS.encoder_units, momentum=0.05, eps=1e-3)

  def forward(self, x):
    x = self.embedding(x)
    if self.emb_batchnorm:
      x = self.batch_norm(x)
    return x

In [14]:
class SqueezeformerEncoder(nn.Module):

  def __init__(self):
    super().__init__()
    self.embedding = SimpleEmbedding()
    attn_dropout, ff_dropout, conv_dropout = FLAGS.attn_drop, FLAGS.ff_drop, FLAGS.conv_drop
    self.encoder = Squeezeformer(
        dim=FLAGS.encoder_units,
        depth=FLAGS.encoder_layers,
        dim_head=FLAGS.mhatt_dimhead,
        heads=FLAGS.mhatt_heads,
        ff_mult=FLAGS.ff_mult,
        conv_expansion_factor=FLAGS.conv1d_expansion_factor,  # 2
        conv_kernel_size=FLAGS.conv1d_ksize_vals[0],
        attn_dropout=attn_dropout,
        ff_dropout=ff_dropout,
        conv_dropout=conv_dropout)

  def forward(self, x):
    x = self.embedding(x)
    x = self.encoder(x)
    return x

In [15]:
class AvgPoolingModule(nn.Module):
  def __init__(self, pool_size, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.pooling = nn.AvgPool1d(pool_size)
    
  def forward(self, x):
    x = x.permute(0, 2, 1)
    x = self.pooling(x)
    x = x.permute(0, 2, 1)
    return x

In [16]:
class Encoder(nn.Module):

  def __init__(self):
    super().__init__()
    self.encoder = SqueezeformerEncoder()
    if FLAGS.encode_pool_size > 1:
      self.pooling = AvgPoolingModule(FLAGS.encode_pool_size)

  def forward(self, frames):
    x = self.encoder(frames)  
    if FLAGS.encode_pool_size > 1:
      x = self.pooling(x)
    return x

In [17]:
class InferModel(nn.Module):
  def __init__(self, model, **kwargs):
    super().__init__(**kwargs)
    self.model = model
  
  def forward(self, frames):
    res = self.model.infer(frames)
    return res
  
  def infer(self, frames):
    return self.forward(frames)

In [18]:
class Model(nn.Module):

  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.encoder = Encoder()
    self.classifer = nn.Sequential(
            nn.Dropout(FLAGS.cls_drop),
            nn.Linear(FLAGS.encoder_units, get_vocab_size()),
        )
      
  # TODO check training flag ok
  def encode(self, frames):
    return self.encoder(frames)

  def forward_(self, frames):
    x = self.encode(frames)
    x = self.classifer(x)
    return x

  def forward(self, inputs):
    if self.training:
      self.input_ = inputs
    x = self.forward_(inputs['frames'])
    res = {
      'pred': x,
    }
    return res
  
  def infer(self, frames):
    return self.forward_(frames)

  def get_loss_fn(self):  
    def ctc_loss(loss_obj, preds, labels, labels_lengths, weights=None):
      preds = F.log_softmax(preds, dim=-1)
      preds_lengths = torch.sum(torch.ones_like(preds[:,:,0]).long(), dim=-1)
      loss = loss_obj(preds.transpose(0, 1), labels, preds_lengths, labels_lengths)
      if weights is not None:
        loss = torch.mean(loss * weights)
      return loss
      
    def loss_fn(res, labels, x, step=None, epoch=None, training=None):
      scalars = {}
      weights = None
      reduction = 'mean'
      if FLAGS.mix_sup:
        weights = x['weight']
        reduction = 'none'
      loss_obj = nn.CTCLoss(zero_infinity=True, reduction=reduction)
      preds = res['pred'].float()
      labels = labels.float()
      labels_lengths = torch.sum((labels != PAD_IDX).long(), dim=-1)
      loss = ctc_loss(loss_obj, preds, labels, labels_lengths, weights)      
      return loss

    return loss_fn
  
  def get_infer_model(self):
    return InferModel(self)

# Code for squeezeformer, notice most code copy/modify from NEMO, and for ROPE encoding copy from huggingface llma part

In [19]:
# simplfiy from NEMO
class TimeReductionModule(nn.Module):
    """
    Squeezeformer Time Reduction procedure. Downsamples the audio by `stride` in the time dimension.

    Args:
        d_model (int): input dimension of MultiheadAttentionMechanism and PositionwiseFeedForward
        out_dim (int): Output dimension of the module.
        kernel_size (int): Conv kernel size for depthwise convolution in convolution module
        stride (int): Downsampling factor in time dimension.
    """

    def __init__(self, d_model: int, out_dim: int, kernel_size: int = 5, stride: int = 2):
        super().__init__()

        self.d_model = d_model
        self.out_dim = out_dim
        self.kernel_size = kernel_size
        self.stride = stride
        ## NOTICE modify here so can dived by 2...
        # self.padding = max(0, self.kernel_size - self.stride) 
        ##  # like k=5, stride=2 here padding is 2 which make 320 -> 160 -> 80
        self.padding = (self.kernel_size + 1) // self.stride - 1

        self.dw_conv = nn.Conv1d(
            in_channels=d_model,
            out_channels=d_model,
            kernel_size=kernel_size,
            stride=stride,
            padding=self.padding,
            groups=d_model,
        )

        self.pw_conv = nn.Conv1d(
            in_channels=d_model, out_channels=out_dim, kernel_size=1, stride=1, padding=0, groups=1,
        )

        self.reset_parameters()

    def forward(self, x):
        x = x.transpose(1, 2)  # [B, C, T]
        x = self.dw_conv(x)
        x = self.pw_conv(x)
        x = x.transpose(1, 2)  # [B, T, C]
        return x

    def reset_parameters(self):
        dw_max = self.kernel_size ** -0.5
        pw_max = self.d_model ** -0.5

        with torch.no_grad():
            torch.nn.init.uniform_(self.dw_conv.weight, -dw_max, dw_max)
            torch.nn.init.uniform_(self.dw_conv.bias, -dw_max, dw_max)
            torch.nn.init.uniform_(self.pw_conv.weight, -pw_max, pw_max)
            torch.nn.init.uniform_(self.pw_conv.bias, -pw_max, pw_max)

In [20]:
class Swish(nn.SiLU):
    """
    Swish activation function introduced in 'https://arxiv.org/abs/1710.05941'
    Mathematically identical to SiLU. See note in nn.SiLU for references.
    """
    
    
class CausalConv1D(nn.Conv1d):
    """
    A causal version of nn.Conv1d where each step would have limited access to locations on its right or left
    All arguments are the same as nn.Conv1d except padding.

    If padding is set None, then paddings are set automatically to make it a causal convolution where each location would not see any steps on its right.

    If padding is set as a list (size of 2), then padding[0] would be used as left padding and padding[1] as right padding.
    It would make it possible to control the number of steps to be accessible on the right and left.
    This mode is not supported when stride > 1. padding[0]+padding[1] should be equal to (kernel_size - 1).
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int,
        stride: int = 1,
        padding: Union[str, int] = 0,
        dilation: int = 1,
        groups: int = 1,
        bias: bool = True,
        padding_mode: str = 'zeros',
        device=None,
        dtype=None,
    ) -> None:
        self.cache_drop_size = None
        if padding is None:
            self._left_padding = kernel_size - 1
            self._right_padding = stride - 1
        else:
            if stride != 1 and padding != kernel_size - 1:
                raise ValueError("No striding allowed for non-symmetric convolutions!")
            if isinstance(padding, int):
                self._left_padding = padding
                self._right_padding = padding
            elif isinstance(padding, list) and len(padding) == 2 and padding[0] + padding[1] == kernel_size - 1:
                self._left_padding = padding[0]
                self._right_padding = padding[1]
            else:
                raise ValueError(f"Invalid padding param: {padding}!")

        self._max_cache_len = self._left_padding

        super(CausalConv1D, self).__init__(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias,
            padding_mode=padding_mode,
            device=device,
            dtype=dtype,
        )

    def update_cache(self, x, cache=None):
        if cache is None:
            new_x = F.pad(x, pad=(self._left_padding, self._right_padding))
            next_cache = cache
        else:
            new_x = F.pad(x, pad=(0, self._right_padding))
            new_x = torch.cat([cache, new_x], dim=-1)
            if self.cache_drop_size > 0:
                next_cache = new_x[:, :, : -self.cache_drop_size]
            else:
                next_cache = new_x
            next_cache = next_cache[:, :, -cache.size(-1) :]
        return new_x, next_cache

    def forward(self, x, cache=None):
        x, cache = self.update_cache(x, cache=cache)
        x = super().forward(x)
        if cache is None:
            return x
        else:
            return x, cache

class ConformerFeedForward(nn.Module):
    """
    feed-forward module of Conformer model.
    """

    def __init__(self, d_model, d_ff, dropout, activation=Swish()):
        super(ConformerFeedForward, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.linear1 = nn.Linear(d_model, d_ff)
        self.activation = activation
        self.dropout = nn.Dropout(p=dropout)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

    def reset_parameters_ff(self):
        ffn1_max = self.d_model ** -0.5
        ffn2_max = self.d_ff ** -0.5
        with torch.no_grad():
            nn.init.uniform_(self.linear1.weight, -ffn1_max, ffn1_max)
            nn.init.uniform_(self.linear1.bias, -ffn1_max, ffn1_max)
            nn.init.uniform_(self.linear2.weight, -ffn2_max, ffn2_max)
            nn.init.uniform_(self.linear2.bias, -ffn2_max, ffn2_max)

activation_registry = {
    "identity": nn.Identity,
    "hardtanh": nn.Hardtanh,
    "relu": nn.ReLU,
    "selu": nn.SELU,
    "swish": nn.SiLU,
    "silu": nn.SiLU,
    "gelu": nn.GELU,
}

class ConformerConvolution(nn.Module):
    """The convolution module for the Conformer model.
    Args:
        d_model (int): hidden dimension
        kernel_size (int): kernel size for depthwise convolution
        pointwise_activation (str): name of the activation function to be used for the pointwise conv.
            Note that Conformer uses a special key `glu_` which is treated as the original default from
            the paper.
    """

    def __init__(
        self, d_model, kernel_size, norm_type='batch_norm', conv_context_size=None, pointwise_activation='glu_'
    ):
        super(ConformerConvolution, self).__init__()
        assert (kernel_size - 1) % 2 == 0
        self.d_model = d_model
        self.kernel_size = kernel_size
        self.norm_type = norm_type

        if conv_context_size is None:
            conv_context_size = (kernel_size - 1) // 2

        if pointwise_activation in activation_registry:
            self.pointwise_activation = activation_registry[pointwise_activation]()
            dw_conv_input_dim = d_model * 2

            if hasattr(self.pointwise_activation, 'inplace'):
                self.pointwise_activation.inplace = True
        else:
            self.pointwise_activation = pointwise_activation
            dw_conv_input_dim = d_model

        self.pointwise_conv1 = nn.Conv1d(
            in_channels=d_model, out_channels=d_model * 2, kernel_size=1, stride=1, padding=0, bias=True
        )

        self.depthwise_conv = CausalConv1D(
            in_channels=dw_conv_input_dim,
            out_channels=dw_conv_input_dim,
            kernel_size=kernel_size,
            stride=1,
            padding=conv_context_size,
            groups=dw_conv_input_dim,
            bias=True,
        )

        if norm_type == 'batch_norm':
            self.batch_norm = nn.BatchNorm1d(dw_conv_input_dim)
        elif norm_type == 'instance_norm':
            self.batch_norm = nn.InstanceNorm1d(dw_conv_input_dim)
        elif norm_type == 'layer_norm':
            self.batch_norm = nn.LayerNorm(dw_conv_input_dim)
        elif norm_type.startswith('group_norm'):
            num_groups = int(norm_type.replace("group_norm", ""))
            self.batch_norm = nn.GroupNorm(num_groups=num_groups, num_channels=d_model)
        else:
            raise ValueError(f"conv_norm_type={norm_type} is not valid!")

        self.activation = Swish()
        self.pointwise_conv2 = nn.Conv1d(
            in_channels=dw_conv_input_dim, out_channels=d_model, kernel_size=1, stride=1, padding=0, bias=True
        )

    def forward(self, x, pad_mask=None, cache=None):
        x = x.transpose(1, 2)
        x = self.pointwise_conv1(x)

        # Compute the activation function or use GLU for original Conformer
        if self.pointwise_activation == 'glu_':
            x = nn.functional.glu(x, dim=1)
        else:
            x = self.pointwise_activation(x)

        if pad_mask is not None:
            x = x.float().masked_fill(pad_mask.unsqueeze(1), 0.0)

        x = self.depthwise_conv(x, cache=cache)
        if cache is not None:
            x, cache = x

        if self.norm_type == "layer_norm":
            x = x.transpose(1, 2)
            x = self.batch_norm(x)
            x = x.transpose(1, 2)
        else:
            x = self.batch_norm(x)

        x = self.activation(x)
        x = self.pointwise_conv2(x)
        x = x.transpose(1, 2)
        if cache is None:
            return x
        else:
            return x, cache

    def reset_parameters_conv(self):
        pw1_max = pw2_max = self.d_model ** -0.5
        dw_max = self.kernel_size ** -0.5

        with torch.no_grad():
            nn.init.uniform_(self.pointwise_conv1.weight, -pw1_max, pw1_max)
            nn.init.uniform_(self.pointwise_conv1.bias, -pw1_max, pw1_max)
            nn.init.uniform_(self.pointwise_conv2.weight, -pw2_max, pw2_max)
            nn.init.uniform_(self.pointwise_conv2.bias, -pw2_max, pw2_max)
            nn.init.uniform_(self.depthwise_conv.weight, -dw_max, dw_max)
            nn.init.uniform_(self.depthwise_conv.bias, -dw_max, dw_max)
            
class ScaleBiasLayer(nn.Module):
    """
    Computes an affine transformation y = x * scale + bias, either learned via adaptive weights, or fixed.
    Efficient alternative to LayerNorm where we can avoid computing the mean and variance of the input, and
    just rescale the output of the previous layer.

    Args:
        d_model (int): input dimension of layer.
        adaptive_scale (bool): whether to learn the affine transformation parameters or not. If set to False,
            the scale is fixed to 1 and bias to 0, effectively performing a No-Op on the input.
            This is done for export compatibility.
    """

    def __init__(self, d_model: int, adaptive_scale: bool):
        super().__init__()
        self.adaptive_scale = adaptive_scale
        if adaptive_scale:
            self.scale = nn.Parameter(torch.ones(d_model))
            self.bias = nn.Parameter(torch.zeros(d_model))
        else:
            self.register_buffer('scale', torch.ones(d_model), persistent=True)
            self.register_buffer('bias', torch.zeros(d_model), persistent=True)

    def forward(self, x):
        scale = self.scale.view(1, 1, -1)
        bias = self.bias.view(1, 1, -1)
        return x * scale + bias

class DepthWiseConv1d(nn.Module):

  def __init__(self, chan_in, chan_out, kernel_size, padding):
    super().__init__()
    self.padding = padding
    self.conv = nn.Conv1d(chan_in, chan_out, kernel_size, groups=chan_in)

  def forward(self, x):
    x = F.pad(x, self.padding)
    return self.conv(x)
  
# attention, feedforward, and conv module


class Scale(nn.Module):

  def __init__(self, scale, fn):
    super().__init__()
    self.fn = fn
    self.scale = scale

  def forward(self, x, **kwargs):
    return self.fn(x, **kwargs) * self.scale


class PreNorm(nn.Module):

  def __init__(self, dim, fn):
    super().__init__()
    self.fn = fn
    self.norm = nn.LayerNorm(dim)

  def forward(self, x, **kwargs):
    x = self.norm(x)
    return self.fn(x, **kwargs)

ROPE code from huggingface llma

In [21]:
# rotary positional embedding
# https://arxiv.org/abs/2104.09864

# https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py
class LlamaRotaryEmbedding(torch.nn.Module):
    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
        super().__init__()

        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.base = base
        inv_freq = 1.0 / (self.base ** (torch.arange(0, self.dim, 2).float().to(device) / self.dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)

        # Build here to make `torch.jit.trace` work.
        self._set_cos_sin_cache(
            seq_len=max_position_embeddings, device=self.inv_freq.device, dtype=torch.get_default_dtype()
        )

    def _set_cos_sin_cache(self, seq_len, device, dtype):
        self.max_seq_len_cached = seq_len
        t = torch.arange(self.max_seq_len_cached, device=device, dtype=self.inv_freq.dtype)

        # freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        freqs = torch.outer(t, self.inv_freq)
        
        # Different from paper, but it uses a different permutation in order to obtain the same calculation
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos()[None, None, :, :].to(dtype), persistent=False)
        self.register_buffer("sin_cached", emb.sin()[None, None, :, :].to(dtype), persistent=False)

    def forward(self, x, seq_len=None):
        # x: [bs, num_attention_heads, seq_len, head_size]
        if seq_len > self.max_seq_len_cached:
            self._set_cos_sin_cache(seq_len=seq_len, device=x.device, dtype=x.dtype)

        return (
            self.cos_cached[:, :, :seq_len, ...].to(dtype=x.dtype),
            self.sin_cached[:, :, :seq_len, ...].to(dtype=x.dtype),
        )


class LlamaLinearScalingRotaryEmbedding(LlamaRotaryEmbedding):
    """LlamaRotaryEmbedding extended with linear scaling. Credits to the Reddit user /u/kaiokendev"""

    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None, scaling_factor=1.0):
        self.scaling_factor = scaling_factor
        super().__init__(dim, max_position_embeddings, base, device)

    def _set_cos_sin_cache(self, seq_len, device, dtype):
        self.max_seq_len_cached = seq_len
        t = torch.arange(self.max_seq_len_cached, device=device, dtype=self.inv_freq.dtype)
        t = t / self.scaling_factor

        # freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        freqs = torch.outer(t, self.inv_freq)
        # Different from paper, but it uses a different permutation in order to obtain the same calculation
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos()[None, None, :, :].to(dtype), persistent=False)
        self.register_buffer("sin_cached", emb.sin()[None, None, :, :].to(dtype), persistent=False)


class LlamaDynamicNTKScalingRotaryEmbedding(LlamaRotaryEmbedding):
    """LlamaRotaryEmbedding extended with Dynamic NTK scaling. Credits to the Reddit users /u/bloc97 and /u/emozilla"""

    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None, scaling_factor=1.0):
        self.scaling_factor = scaling_factor
        super().__init__(dim, max_position_embeddings, base, device)

    def _set_cos_sin_cache(self, seq_len, device, dtype):
        self.max_seq_len_cached = seq_len

        if seq_len > self.max_position_embeddings:
            base = self.base * (
                (self.scaling_factor * seq_len / self.max_position_embeddings) - (self.scaling_factor - 1)
            ) ** (self.dim / (self.dim - 2))
            inv_freq = 1.0 / (base ** (torch.arange(0, self.dim, 2).float().to(device) / self.dim))
            self.register_buffer("inv_freq", inv_freq, persistent=False)

        t = torch.arange(self.max_seq_len_cached, device=device, dtype=self.inv_freq.dtype)

        # freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        freqs = torch.outer(t, self.inv_freq)
        # Different from paper, but it uses a different permutation in order to obtain the same calculation
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos()[None, None, :, :].to(dtype), persistent=False)
        self.register_buffer("sin_cached", emb.sin()[None, None, :, :].to(dtype), persistent=False)


def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    # return torch.cat((-x2, x1), dim=-1)
    # return torch.cat((torch.neg(x2), x1), dim=-1)
    return torch.cat((x2 * (-1.), x1), dim=-1)

def apply_rotary_pos_emb(q, k, cos, sin, position_ids):
    # The first two dimensions of cos and sin are always 1, so we can `squeeze` them.
    cos = cos.squeeze(1).squeeze(0)  # [seq_len, dim]
    sin = sin.squeeze(1).squeeze(0)  # [seq_len, dim]
    cos = cos[position_ids].unsqueeze(1)  # [bs, 1, seq_len, dim]
    sin = sin[position_ids].unsqueeze(1)  # [bs, 1, seq_len, dim]
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed

Attention is important which use ROPE relative pos encodeing

In [22]:
class Attention(nn.Module):
  def __init__(self, dim, dim_head=64, 
                heads=8, dropout=0., max_pos_emb=512, 
                relpos_att=True, rope=False):
    super().__init__()
    self.scale = dim_head ** -0.5
    self.heads = heads
    inner_dim = heads * dim_head

    self.dropout = nn.Dropout(dropout)
    self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
    self.to_out = nn.Linear(inner_dim, dim, bias = False)
    self.max_position_embeddings = max_pos_emb
    self.dim_head = dim_head
    self.relpos_att = relpos_att
    self.rope = rope
    if relpos_att:
      if rope:
        self._init_rope()
      else:
        self.rel_pos_emb = nn.Embedding(2 * max_pos_emb + 1, dim_head)

  def _init_rope(self):
    scaling_type = FLAGS.scaling_type
    if scaling_type is None:
        self.rotary_emb = LlamaRotaryEmbedding(self.dim_head, max_position_embeddings=self.max_position_embeddings)
    else:
        scaling_factor = FLAGS.scaling_factor
        if scaling_type == "linear":
            self.rotary_emb = LlamaLinearScalingRotaryEmbedding(
                self.dim_head, max_position_embeddings=self.max_position_embeddings, scaling_factor=scaling_factor
            )
        elif scaling_type == "dynamic":
            self.rotary_emb = LlamaDynamicNTKScalingRotaryEmbedding(
                self.dim_head, max_position_embeddings=self.max_position_embeddings, scaling_factor=scaling_factor
            )
        else:
            raise ValueError(f"Unknown RoPE scaling type {scaling_type}")
      

  def forward(self, x):
    n, device, h = x.shape[-2], x.device, self.heads

    q, k, v = self.to_qkv(x).chunk(3, dim = -1)
    
    q = q.view(q.shape[0], q.shape[1], h, -1).permute(0, 2, 1, 3)
    k = k.view(k.shape[0], k.shape[1], h, -1).permute(0, 2, 1, 3)
    v = v.view(v.shape[0], v.shape[1], h, -1).permute(0, 2, 1, 3)

    if self.relpos_att:
      cos, sin = self.rotary_emb(v, seq_len=n)
      position_ids = torch.arange(0, n, dtype=torch.long, device=device)
      position_ids = position_ids.unsqueeze(0).view(-1, n)
      q, k = apply_rotary_pos_emb(q, k, cos, sin, position_ids)
      sim = torch.matmul(q, k.transpose(2, 3)) * self.scale
    else:
      sim = torch.matmul(q, k.permute(0, 1, 3, 2)) * self.scale
    
    attn = F.softmax(sim, dim=-1, dtype=torch.float32).to(q.dtype)
    attn = self.dropout(attn)
    
    out = torch.matmul(attn, v)
    out = out.permute(0, 2, 1, 3)
    out = out.reshape(out.shape[0], out.shape[1], -1)
    out = self.to_out(out)
    out = self.dropout(out)
    return out

In [23]:
class SqueezeformerBlock(nn.Module):

  def __init__(self,
               *,
               dim,
               dim_head=64,
               heads=8,
               ff_mult=4,
               conv_expansion_factor=2,
               conv_kernel_size=31,
               attn_dropout=0.,
               ff_dropout=0.,
               conv_dropout=0.,
               conv_causal=False,
               relpos_att=True,
               rope=False,
               inst_drop=None,
               skip_factor=None):
    super().__init__()
    # first feed forward module
    self.norm_feed_forward1 = nn.LayerNorm(dim)
    self.feed_forward1 = ConformerFeedForward(d_model=dim, d_ff=dim * ff_mult, dropout=ff_dropout)
    self.feed_forward1_scale = ScaleBiasLayer(d_model=dim, adaptive_scale=True)
    
    # convolution module
    self.norm_conv = nn.LayerNorm(dim)
    self.conv = ConformerConvolution(
        d_model=dim, kernel_size=conv_kernel_size, norm_type='batch_norm', pointwise_activation='swish'
    )
    self.conv_scale = ScaleBiasLayer(d_model=dim, adaptive_scale=True)
    
    # multi-headed self-attention module
    self.norm_self_att = nn.LayerNorm(dim)
    self.self_attn = Attention(dim=dim,
                          dim_head=dim_head,
                          heads=heads,
                          dropout=attn_dropout,
                          max_pos_emb=FLAGS.n_frames,
                          relpos_att=relpos_att,
                          rope=rope)
    self.self_attn_scale = ScaleBiasLayer(d_model=dim, adaptive_scale=True)

    # second feed forward module
    self.norm_feed_forward2 = nn.LayerNorm(dim)
    self.feed_forward2 = ConformerFeedForward(d_model=dim, d_ff=dim * ff_mult, dropout=ff_dropout)
    self.feed_forward2_scale = ScaleBiasLayer(d_model=dim, adaptive_scale=True)
    
    self.inst_drop = inst_drop if inst_drop is not None else FLAGS.inst_drop_rate
    # 0.2
    self.dropout = InstanceDropout(self.inst_drop)
    self.skip_factor = skip_factor if skip_factor is not None else FLAGS.skip_factor
    
    self.fc_factor = 0.5
    
    self.reset_parameters()
    
  def reset_parameters(self):
    # Used for Squeezeformer initialization only
    self.feed_forward1.reset_parameters_ff()
    self.feed_forward2.reset_parameters_ff()
    self.conv.reset_parameters_conv()

  def forward(self, x):
    residual = x
    x = self.self_attn_scale(x)
    x = self.self_attn(x)
    x = residual + self.dropout(x) * self.skip_factor
    
    x = self.norm_self_att(x)
    residual = x
    x = self.feed_forward1_scale(x)
    x = self.feed_forward1(x)
    x = residual + self.dropout(x) * self.skip_factor * self.fc_factor
    x = self.norm_feed_forward1(x)
    residual = x

    x = self.conv_scale(x)
    x = self.conv(x)
    x = residual + self.dropout(x) * self.skip_factor
    x = self.norm_conv(x)
    residual = x
    
    x = self.feed_forward2_scale(x)
    x = self.feed_forward2(x)
    x = residual + self.dropout(x) * self.skip_factor * self.fc_factor
    x = self.norm_feed_forward2(x)

    return x


class Squeezeformer(nn.Module):

  def __init__(self,
               dim,
               *,
               depth,
               dim_head=64,
               heads=8,
               ff_mult=4,
               conv_expansion_factor=2,
               conv_kernel_size=31,
               attn_dropout=0.,
               ff_dropout=0.,
               conv_dropout=0.,
               conv_causal=False):
    super().__init__()
    self.dim = dim
    self.layers = nn.ModuleList([])
    self.inst_drops = [None] * depth
    heads_ = heads
    conv_kernel_size_ = conv_kernel_size
    for i in range(depth):    
      relpos_att = True
      dim_head_ = dim_head
      rope = True
      heads_ = heads
      if i < FLAGS.time_reduce_idx:
        heads_ = heads // 2
      
      self.layers.append(SqueezeformerBlock(dim=dim,
                          dim_head=dim_head_,
                          heads=heads_,
                          ff_mult=ff_mult,
                          conv_expansion_factor=conv_expansion_factor,
                          conv_kernel_size=conv_kernel_size_,
                          conv_causal=conv_causal,
                          attn_dropout=attn_dropout,
                          ff_dropout=ff_dropout,
                          conv_dropout=conv_dropout,
                          relpos_att=relpos_att,
                          rope=rope,
                          inst_drop=self.inst_drops[i]))
      
    if FLAGS.time_reduce:
      if FLAGS.time_reduce_idx < depth - 1:
        reduction_module = TimeReductionModule(dim, dim, kernel_size=FLAGS.time_kernel_size, stride=FLAGS.time_stride) 
        self.layers.insert(FLAGS.time_reduce_idx, reduction_module)
      
  def forward(self, x):
    for i, layer in enumerate(self.layers):
      x = layer(x)
    return x

In [24]:
model = Model()
model

Model(
  (encoder): Encoder(
    (encoder): SqueezeformerEncoder(
      (embedding): SimpleEmbedding(
        (embedding): Linear(in_features=769, out_features=200, bias=False)
        (batch_norm): BatchNorm(
          (bn): BatchNorm1d(200, eps=0.001, momentum=0.05, affine=True, track_running_stats=True)
        )
      )
      (encoder): Squeezeformer(
        (layers): ModuleList(
          (0-6): 7 x SqueezeformerBlock(
            (norm_feed_forward1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
            (feed_forward1): ConformerFeedForward(
              (linear1): Linear(in_features=200, out_features=800, bias=True)
              (activation): Swish()
              (dropout): Dropout(p=0, inplace=False)
              (linear2): Linear(in_features=800, out_features=200, bias=True)
            )
            (feed_forward1_scale): ScaleBiasLayer()
            (norm_conv): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
            (conv): ConformerConvolution(


In [25]:
from torchinfo import summary
input_shape = (1, FLAGS.n_frames, get_n_cols())
summary(model.get_infer_model(), input_shape)

Layer (type:depth-idx)                                                                Output Shape              Param #
InferModel                                                                            [1, 80, 61]               --
├─Model: 1-1                                                                          --                        --
│    └─Encoder: 2-1                                                                   [1, 80, 200]              --
│    │    └─SqueezeformerEncoder: 3-1                                                 [1, 160, 200]             16,781,400
│    │    └─AvgPoolingModule: 3-2                                                     [1, 80, 200]              --
│    └─Sequential: 2-2                                                                [1, 80, 61]               --
│    │    └─Dropout: 3-3                                                              [1, 80, 200]              --
│    │    └─Linear: 3-4                                            

# TODO you may load weights of trained torch model here at first

In [26]:
# TFLite model for submission
class TFLiteModel(tf.keras.Model):

  def __init__(self, model):
    super(TFLiteModel, self).__init__()

    # Load the feature generation and main models
    self.preprocess_layer = PreprocessLayer(FLAGS.n_frames)
    self.model = model

  @tf.function(jit_compile=True)
  def infer(self, frames):
    return self.model(frames)

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None, N_COLS], dtype=tf.float32, name='inputs')
  ])
  def call(self, inputs):
    # Preprocess Data
    frames_inp = self.preprocess_layer(inputs)
    # x = frames_inp
    # Add Batch Dimension
    frames_inp = tf.expand_dims(frames_inp, axis=0)

    outputs = self.infer(frames_inp)
    # y = outputs
 
    # Squeeze outputs
    outputs = tf.squeeze(outputs, axis=0)
    outputs = decode_phrase(outputs)
    
    # for 0 is PAD_IDX
    outputs -= 1
    # outputs = tf.one_hot(outputs, get_vocab_size())
    # vocab_size = 61 if not FLAGS.no_eos else 60
    vocab_size = get_vocab_size()
    outputs = tf.one_hot(outputs, vocab_size)
    if FLAGS.decode_phrase_type:
      ouputs = outputs[1:]

    # Return a dictionary with the output tensor
    return {'outputs': outputs}
    # return {
    #   'outputs': outputs,
    #   'frames': x,
    #   'intermediate': y,
    # }

  # TODO not work... an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.
  def get_model(self):
    inputs = tf.keras.layers.Input([N_COLS],
                                    dtype=tf.float32,
                                    name='inputs')
    out = self.call(inputs)
    model = tf.keras.models.Model(inputs, out)
    model.summary()
    return model

In [27]:
FLAGS.model_dir = './'

def get_tflite_model(model):
  tflite_keras_model = TFLiteModel(model)
  return tflite_keras_model

def to_tflite_model(model):
  tflite_keras_model = get_tflite_model(model)
  # tflite_keras_model = tflite_keras_model.get_model()
  # plot_model(tflite_func_model, to_file=f'{FLAGS.model_dir}/tflite.png', show_shapes=True, show_layer_names=True)
  ic(tflite_keras_model)
  # Create Model Converter
  converter = tf.lite.TFLiteConverter.from_keras_model(tflite_keras_model)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.target_spec.supported_types = [tf.float16]
  #  1/4 size but seems hurt acc a lot like 0.8 to 0.7
  # converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8,
  #                                        tf.lite.OpsSet.TFLITE_BUILTINS]
  converter._experimental_default_to_single_batch_in_tensor_list_ops = True
  
  # converter.post_training_quantize = True
  ic(converter)
  ## converter.experimental_new_converter = True
  # Convert Model
  tflite_model = converter.convert()
  # ic(tflite_model)
  # Write Model
  with open(f'{FLAGS.model_dir}/model.tflite', 'wb') as f:
    f.write(tflite_model)

  os.system(f'du -h {FLAGS.model_dir}/model.tflite')

In [28]:
!pip install sty
sys.path.append('../input/aslfr-nobuco/')
import nobuco

In [29]:
FLAGS.convert_trace = False

In [ ]:
def torch2keras(model):
  import nobuco
  from nobuco import ChannelOrder, ChannelOrderingStrategy
  from nobuco.layers.weight import WeightLayer
  input_shape = [1, FLAGS.n_frames, get_n_cols()]
  ic(input_shape)
  dummy_input = torch.randn(input_shape)
  model = model.get_infer_model()

  # model = model.half().float()
  model = model.to('cpu')
  model = model.eval()
  
  ## TODO help merge to nobuco codebase
  # for this you could use x = F.avg_pool1d(x, FLAGS.encode_pool_size)
  @nobuco.converter(F.avg_pool1d, channel_ordering_strategy=ChannelOrderingStrategy.FORCE_TENSORFLOW_ORDER)
  def converter_avg_pool1d(input: torch.Tensor, input2: int, inplace: bool = False):
    return lambda input, input2, inplace=False: tf.keras.layers.AveragePooling1D(input2)(input)
  
  @nobuco.converter(nn.AvgPool1d, channel_ordering_strategy=ChannelOrderingStrategy.FORCE_TENSORFLOW_ORDER)
  def converter_AvgPool1d(self, input: torch.Tensor):
    return tf.keras.layers.AveragePooling1D(self.kernel_size)
    
  @nobuco.converter(torch.ones_like, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
  def converter_ones_like(input, *args, **kwargs):
      def func(input, *args, **kwargs):
          return tf.ones_like(input)
      return func
  
  @nobuco.converter(torch.Tensor.bmm, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS_OR_PYTORCH, autocast=True)
  def converter_bmm(self, value, *args, **kwargs):
    def func(self, value, *args, **kwargs):
        return tf.matmul(self, value)
    return func
  
  @nobuco.converter(torch.flip, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
  def converter_flip(input, input2, *args, **kwargs):
      def func(input, input2, *args, **kwargs):
          return tf.reverse(input, input2)
      return func
  
  
  input_shape[0] = None
  keras_model = nobuco.pytorch_to_keras(
      model,
      args=[dummy_input], 
      input_shapes={dummy_input: input_shape},
      inputs_channel_order=ChannelOrder.PYTORCH,
      outputs_channel_order=ChannelOrder.PYTORCH,
      trace_shape=True, 
      debug_traces=FLAGS.convert_trace,
    )
  keras_model.summary()
  return keras_model
  

# Convert torch to keras using nobuco  

In [31]:
%%capture
keras_model = torch2keras(model)

In [32]:
FLAGS.pad_thre = 0.3
FLAGS.decode_phrase_type = False
to_tflite_model(keras_model)

ic| self.n_cols: 769
ic| self.means.shape: (384,), self.stds.shape: (384,)
ic| tflite_keras_model: <__main__.TFLiteModel object at 0x7a86aa2576a0>
ic| converter: <tensorflow.lite.python.lite.TFLiteKerasModelConverterV2 object at 0x7a864cfa2410>


33M	.//model.tflite
